In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import env

In [2]:
''' function to connect to CodeUp SQL database'''
def get_connection(db, user=env.username, host=env.host, password=env.password):
    return f'mysql+pymysql://{env.username}:{env.password}@{env.host}/{db}'

In [3]:
def get_logs():
    ''' function to acquire the curriculum logs data from MySQL server with the columns i wanted to be dataframe.
    And renamed the columns in the  SQL querry for convienience'''
    query = '''
       SELECT logs.date,  logs.time,
       logs.path as endpoint,
       logs.user_id as user,
       logs.ip as source_ip,
       cohorts.name as cohort_name,
       cohorts.start_date as start_date,
       cohorts.end_date as end_date,
       cohorts.program_id as program_id
       FROM logs
       JOIN cohorts ON logs.cohort_id= cohorts.id;
         '''
    
    
    df= pd.read_sql(query, get_connection('curriculum_logs'))
    
    return df

In [7]:
def prepare_log(df):
    ''' This prepare function set the date column as index, drop unwanted columns\
    and set the start date and end date to date time format'''
    #change the date column to datetime
    df['date']=pd.to_datetime(df.date)
    # set date column to index
    df = df.set_index(df.date)
    #set the start_date and end_date column to datetime format
    df.start_date = pd.to_datetime(df.start_date)
    df.end_date = pd.to_datetime(df.end_date)
    #split the endpoint into 4 different sections using / as sepeartor and concatenate to the dataframe
    df= pd.concat([df, df.endpoint.str.split('/',3, expand = True)], axis=1)
    # renaming the columns created after the split of endpoint columns as page 1, page 2,page 3, page 4 respectively
    df.rename(columns={0:'page_1',1:'page_2',2:'page_3',3:'page_4'}, inplace = True)
    # data science program dataframe
    ds_df= df[df.program_id == 3]
    # web developers dataframe
    web_df = df[(df.program_id != 3) & (df.cohort_name != 'Staff')]
    #staff only dataframe
    staff_df = df[df.cohort_name == 'staff']
    return df,ds_df, web_df, staff_df

In [5]:
# test the acquiring function
df=get_logs()
df.head(2)

,date,time,endpoint,user,source_ip,cohort_name,start_date,end_date,program_id
0,2018-01-26,09:55:03,/,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1
1,2018-01-26,09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1


In [8]:
# test the prepare function
df, ds_df, web_df, staff_df= prepare_log(df)


In [9]:
df.head()

,date,time,endpoint,user,source_ip,cohort_name,start_date,end_date,program_id,page_1,page_2,page_3,page_4
date,,,,,,,,,,,,,
2018-01-26,2018-01-26,09:55:03,/,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,,,None,None
2018-01-26,2018-01-26,09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,java-ii,None,None,None
2018-01-26,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,java-ii,object-oriented-programming,None,None
2018-01-26,2018-01-26,09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,slides,object_oriented_programming,None,None
2018-01-26,2018-01-26,09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,2,javascript-i,conditionals,None,None


In [10]:
ds_df.head()

,date,time,endpoint,user,source_ip,cohort_name,start_date,end_date,program_id,page_1,page_2,page_3,page_4
date,,,,,,,,,,,,,
2019-08-20,2019-08-20,09:39:58,/,466,97.105.19.58,Bayes,2019-08-19,2020-01-30,3,,,None,None
2019-08-20,2019-08-20,09:39:59,/,467,97.105.19.58,Bayes,2019-08-19,2020-01-30,3,,,None,None
2019-08-20,2019-08-20,09:39:59,/,468,97.105.19.58,Bayes,2019-08-19,2020-01-30,3,,,None,None
2019-08-20,2019-08-20,09:40:02,/,469,97.105.19.58,Bayes,2019-08-19,2020-01-30,3,,,None,None
2019-08-20,2019-08-20,09:40:08,/,470,97.105.19.58,Bayes,2019-08-19,2020-01-30,3,,,None,None
